In [3]:
import pandas as pd, numpy as np, datetime as dt, re
import matplotlib.pyplot as plt, scipy.stats as stats
import sqlalchemy as sql

from db_info import connection_str

In [2]:
engine = sql.create_engine(connection_str)
conn = engine.connect()

In [4]:
df_odds = pd.read_sql("select * from odds", con=conn, index_col="id")
df_time = pd.read_sql("select * from time", con=conn, index_col="id", parse_dates="timestamp")
df_gamecodes = pd.read_sql("select * from gamecodes", con=conn, index_col="id", parse_dates="date")

In [5]:
df_odds.shape

(87248, 9)

In [6]:
df_time.shape

(763751, 5)

In [7]:
df_gamecodes.shape

(5131, 4)

In [8]:
df_time.head()

,timestamp,book,spread,total,game_id
id,,,,,
1,2021-10-30 08:10:27,BOVADA,-7 -110,+224.5 -110,1459741
2,2021-10-30 08:35:12,BOVADA,-7 -110,+223.5 -110,1459741
3,2021-10-30 15:43:56,BOVADA,-7 -110,+224 -110,1459741
4,2021-10-30 15:50:00,BOVADA,-7 -110,+224.5 -110,1459741
5,2021-10-30 16:06:15,BOVADA,-7 -110,+223.5 -110,1459741


In [9]:
df_gamecodes.head()

,home_abbv,away_abbv,date,game_id
id,,,,
1,UTA,SAC,2021-11-02 21:00:00,1459841
2,DAL,MIA,2021-11-02 19:30:00,1459836
3,CHR,CLE,2021-11-01 19:00:00,1459786
4,SAN,DAL,2021-11-03 20:30:00,1459896
5,ATL,WAS,2021-11-01 19:30:00,1459806


In [10]:
df_gamecodes["game_id"] = df_gamecodes.game_id.astype("str")
df_odds["game_id"] = df_odds.game_id.astype("str")
df_time["game_id"] = df_time.game_id.astype("str")

In [11]:
df_odds

,team_abbv,book,moneyline,spread,spread_odds,total,over_odds,under_odds,game_id
id,,,,,,,,,
1,SAC,Opening,305,10.0,-110,220.0,-110,-110,1459841
2,UTA,Opening,-455,-10.0,-110,220.0,-110,-110,1459841
3,SAC,BOVADA,310,9.0,-110,223.0,-110,-110,1459841
4,UTA,BOVADA,-415,-9.0,-110,223.0,-110,-110,1459841
5,SAC,BetOnline,340,9.0,-106,223.0,-110,-110,1459841
...,...,...,...,...,...,...,...,...,...
87244,DEN,MyBookie,-350,-8.0,-105,213.0,-110,-110,1032068
87245,DET,GTBets,270,7.5,-109,212.5,-109,-109,1032068
87246,DEN,GTBets,-340,-7.5,-109,212.5,-109,-109,1032068


In [18]:
def seasoner(date):
    if date <= dt.datetime(2018, 6, 8):
        return "17-18"
    elif date <= dt.datetime(2019, 6, 13):
        return "18-19"
    elif date <= dt.datetime(2020, 10, 11):
        return "19-20"
    elif date <= dt.datetime(2021, 7, 21):
        return "20-21"
    else:
        return "21-22"

In [19]:
df_gamecodes["season"] = df_gamecodes.date.apply(seasoner)

In [20]:
df_gamecodes.groupby(["season"]).count()

,home_abbv,away_abbv,date,game_id
season,,,,
17-18,1369,1369,1369,1369
18-19,1318,1318,1318,1318
19-20,1144,1144,1144,1144
20-21,1182,1182,1182,1182
21-22,118,118,118,118


In [73]:
engine_2 = sql.create_engine("sqlite:///../data/interim/NBA.db")
conn_2 = engine_2.connect()
df_player_basic= pd.read_sql("SELECT * FROM basic_stats WHERE name = 'Team Totals'", con=conn_2)

In [ ]:
not_in_NBA = merged_df[merged_df.home_team.isnull()].sort_values(['game_code'])[lambda x: x.date < dt.datetime(2021, 10, 17)].game_code.values
not_in_SQL = merged_df[merged_df.home_abbv.isnull()].sort_values("game_code").game_code.unique()

In [ ]:
# feed game_codes into game.py
not_in_NBA

array(['202008150POR', '202012220BRK', '202012220LAL', '202012230BOS',
       '202012230CHI', '202012230CLE', '202012230DEN', '202012230IND',
       '202012230MEM', '202012230MIN', '202012230ORL', '202012230PHI',
       '202012230PHO', '202012230POR', '202105180BOS', '202105180IND',
       '202105190LAL', '202105190MEM', '202105200WAS', '202105210GSW'],
      dtype=object)

In [ ]:
# look up in odds shark and feed into shark.py
not_in_SQL

array(['201801290ATL', '201803180LAC', '201803180MIN', '201803180NOP',
       '201803180TOR', '201803220CHO', '201803220DAL', '201803220HOU',
       '201803220NOP', '201803220ORL', '201803220SAC', '201803250BRK',
       '201803250GSW', '201803250HOU', '201803250IND', '201803250MIL',
       '201803250OKC', '201803250SAC', '201803250TOR', '201803250WAS',
       '201803260CHO', '201803260DET', '201803260MIN', '201803260PHI',
       '201803260PHO', '201803310BOS', '201803310MIA', '201803310NYK',
       '201803310SAC', '201803310WAS', '201804050CLE', '201804050DEN',
       '201804050HOU', '201804050IND', '201804050MIL', '201804050UTA',
       '201804060BOS', '201804060DET', '201804060LAL', '201804060MEM',
       '201804060NYK', '201804060ORL', '201804060PHI', '201804060PHO',
       '201804060TOR', '201804060WAS', '201811190ATL', '201903260CHO',
       '201903260CLE', '201903260DAL', '201903260DEN', '201903260LAL',
       '201903260MIA', '201903260MIL', '201903260MIN', '201903260NOP',
      

In [ ]:
with open("../data/interim/bball_ref_additional.txt", "w") as file:
    for x in not_in_NBA:
        file.write(x + "\n")
    file.close()

In [30]:
df_player_basic.head(20)

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
0,14,Team Totals,240.0,38,92,0.413,12,30,0.400,19,...,6,6,14,17,107,0.0,0,TotalBasics,DAL,201810260TOR
1,28,Team Totals,60.0,9,23,0.391,2,9,0.222,6,...,1,0,2,7,26,0.0,0,1Q,DAL,201810260TOR
2,42,Team Totals,60.0,13,23,0.565,4,7,0.571,4,...,2,3,3,3,34,0.0,0,2Q,DAL,201810260TOR
3,56,Team Totals,120.0,22,46,0.478,6,16,0.375,10,...,3,3,5,10,60,0.0,0,1H,DAL,201810260TOR
4,70,Team Totals,60.0,9,22,0.409,4,8,0.500,7,...,1,2,2,3,29,0.0,0,3Q,DAL,201810260TOR
5,84,Team Totals,60.0,7,24,0.292,2,6,0.333,2,...,2,1,7,4,18,0.0,0,4Q,DAL,201810260TOR
6,98,Team Totals,120.0,16,46,0.348,6,14,0.429,9,...,3,3,9,7,47,0.0,0,2H,DAL,201810260TOR
7,110,Team Totals,240.0,44,91,0.484,11,27,0.407,17,...,10,5,12,22,116,0.0,0,TotalBasics,TOR,201810260TOR
8,122,Team Totals,60.0,13,24,0.542,4,11,0.364,9,...,2,2,2,7,39,0.0,0,1Q,TOR,201810260TOR
9,134,Team Totals,60.0,12,23,0.522,4,6,0.667,2,...,2,1,3,3,30,0.0,0,2Q,TOR,201810260TOR


In [74]:
check_reference = df_player_basic.query("timetype == 'TotalBasics'").loc[:, ["team", "game_code"]]

In [68]:
abbv_regex = re.compile("[A-Z]{3}")

In [75]:
check_reference["home_team"] = check_reference.game_code.apply(lambda x: abbv_regex.search(x)[0])

In [76]:
check_reference["home_team_bin"] = (check_reference["team"] == check_reference["home_team"]).astype("int32")

In [36]:
check_reference

,team,game_code,home_team,home_team_bin
0,DAL,201810260TOR,TOR,0
7,TOR,201810260TOR,TOR,1
14,BOS,201810190TOR,TOR,0
21,TOR,201810190TOR,TOR,1
28,CLE,201810170TOR,TOR,0
...,...,...,...,...
69899,IND,202012230IND,IND,1
69906,MIA,202012230ORL,ORL,0
69913,ORL,202012230ORL,ORL,1
69920,SAS,202012230MEM,MEM,0


In [37]:
df_gamecodes.shape

(5131, 6)

In [38]:
df_gamecodes.home_abbv.unique()

array(['UTA', 'DAL', 'CHR', 'SAN', 'ATL', 'LAL', 'GS', 'IND', 'BKN',
       'MIN', 'SAC', 'CHI', 'WAS', 'CLE', 'LAC', 'BOS', 'MIL', 'PHI',
       'DET', 'ORL', 'PHO', 'MEM', 'MIA', 'NY', 'TOR', 'DEN', 'NOP',
       'POR', 'HOU', 'OKC'], dtype=object)

In [39]:
check_reference.team.unique()

array(['DAL', 'TOR', 'BOS', 'CLE', 'CHO', 'MIN', 'DET', 'NOP', 'NYK',
       'PHI', 'WAS', 'LAC', 'SAS', 'POR', 'BRK', 'IND', 'GSW', 'PHO',
       'OKC', 'ORL', 'CHI', 'UTA', 'HOU', 'ATL', 'MIL', 'DEN', 'SAC',
       'MIA', 'LAL', 'MEM'], dtype=object)

In [40]:
shark_teams_flipper = {'CHR': 'CHO', 'SAN': 'SAS', 'GS': 'GSW', 'BKN': 'BRK', 'NY': 'NYK'}
df_gamecodes.home_abbv = df_gamecodes.home_abbv.apply(lambda x: shark_teams_flipper[x] if x in shark_teams_flipper.keys() else x)
df_gamecodes.away_abbv = df_gamecodes.away_abbv.apply(lambda x: shark_teams_flipper[x] if x in shark_teams_flipper.keys() else x) 

In [46]:
# Transform game_id into bball ref code 
df_gamecodes["game_code"] = (df_gamecodes.date.apply(dt.datetime.strftime, format="%Y%m%d0")) + df_gamecodes.home_abbv

In [77]:
merged_df = df_gamecodes.merge(check_reference, "left", on="game_code")

In [64]:
merged_df[merged_df.season != "21-22"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10059 entries, 117 to 10176
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   home_abbv      10059 non-null  object        
 1   away_abbv      10059 non-null  object        
 2   date           10059 non-null  datetime64[ns]
 3   game_id        10059 non-null  object        
 4   season         10059 non-null  object        
 5   game_code      10059 non-null  object        
 6   team           10052 non-null  object        
 7   home_team      10052 non-null  object        
 8   home_team_bin  10052 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 785.9+ KB


In [78]:
completed_seasons = merged_df[merged_df.season != "21-22"]
completed_seasons[completed_seasons.team.isnull()]

,home_abbv,away_abbv,date,game_id,season,game_code,team,home_team,home_team_bin


In [58]:
check_reference[check_reference.home_team == "GSW"].sort_values("game_code")

,team,game_code,home_team,home_team_bin
56741,GSW,201710170GSW,GSW,1
56734,HOU,201710170GSW,GSW,0
56552,TOR,201710250GSW,GSW,0
56559,GSW,201710250GSW,GSW,1
56867,GSW,201710270GSW,GSW,1
...,...,...,...,...
68345,GSW,202105110GSW,GSW,1
68233,GSW,202105140GSW,GSW,1
68226,NOP,202105140GSW,GSW,0
68282,MEM,202105160GSW,GSW,0
